# Benchmark Model Profiling

This notebook queries the option data, runs the benchmark Black-76 model, and profiles its execution performance.

In [1]:
import sys
import os
import cProfile
import pstats
import io
import time
from pstats import SortKey

# Add project root to sys.path (parent directory of benchmark/)
sys.path.insert(0, os.path.abspath(".."))

from query.csv_loader import load_all_csv_data, filter_by_time_range
from benchmark.benchmark_model import calculate_benchmark

## 1. Load and Filter Data

In [2]:
try:
    df = load_all_csv_data(data_dir="../data")
    print("Successfully loaded.")
    filter_df = filter_by_time_range(df=df, start_time="09:30", end_time="10:00")
except Exception as e:
    print(f"Error loading data: {e}")
    raise e

Successfully loaded.


## 2. Profile Benchmark Model

We use `cProfile` to profile the `calculate_benchmark` function.

In [3]:
print("Starting benchmark execution with profiling...")

# Initialize profiler
pr = cProfile.Profile()
pr.enable()

start_bench = time.time()

# Run the benchmark model
result_df = calculate_benchmark(filter_df)

end_bench = time.time()
pr.disable()

total_time = end_bench - start_bench
print(f"Benchmark completed in {total_time:.4f} seconds.")

Starting benchmark execution with profiling...


Calculating Risks: 100%|██████████| 5948/5948 [00:42<00:00, 140.29it/s]

Benchmark completed in 42.4402 seconds.


## 3. Profiling Results

In [4]:
s = io.StringIO()
sortby = SortKey.CUMULATIVE
ps = pstats.Stats(pr, stream=s).sort_stats(sortby)
ps.print_stats(20)
print(s.getvalue())

         25988175 function calls (25954336 primitive calls) in 42.443 seconds

   Ordered by: cumulative time
   List reduced from 892 to 20 due to restriction <20>

   ncalls  tottime  percall  cumtime  percall filename:lineno(function)
      936    0.174    0.000   52.525    0.056 /Library/Frameworks/Python.framework/Versions/3.14/lib/python3.14/asyncio/base_events.py:1966(_run_once)
     5948    0.290    0.000   36.352    0.006 /Users/jg/Documents/mlp_projects/benchmark/black76_model.py:48(calculate_implied_volatility)
   133246    1.474    0.000   35.455    0.000 /Users/jg/Documents/mlp_projects/benchmark/black76_model.py:15(black76_price)
   284333    8.896    0.000   32.936    0.000 /Users/jg/Documents/mlp_projects/venv/lib/python3.14/site-packages/scipy/stats/_distn_infrastructure.py:2121(cdf)
   302174    2.124    0.000   12.894    0.000 /Users/jg/Documents/mlp_projects/venv/lib/python3.14/site-packages/scipy/stats/_distn_infrastructure.py:609(argsreduce)
     2067    0.048    

## 4. Data Preview

In [5]:
result_df[
    [
        "Date",
        "ExpirationDate",
        "Strike",
        "CallPut",
        "VWAP",
        "ImpliedVol",
        "Delta",
        "Gamma",
        "Vega",
        "Theta",
        "Rho",
    ]
].head()

,Date,ExpirationDate,Strike,CallPut,VWAP,ImpliedVol,Delta,Gamma,Vega,Theta,Rho
0,20200129,20200131,1540.0,P,0.1300,0.958031,-0.003177,0.000073,1.328798,-116.248565,-0.000712
1,20200129,20200131,1787.5,P,8.0667,0.604821,-0.175869,0.003103,35.620446,-1967.550160,-0.044171
2,20200129,20200131,1535.0,P,0.1300,0.972829,-0.003132,0.000071,1.311663,-116.521966,-0.000712
3,20200129,20200131,1760.0,P,3.9457,0.606228,-0.098221,0.002072,23.878473,-1321.975022,-0.021605
4,20200129,20200131,1730.0,P,1.9500,0.625942,-0.052695,0.001246,14.821601,-847.223990,-0.010678
